In [1]:
import numpy
import matplotlib.pyplot as plt
import matplotlib
import scipy.special
import scipy.misc
from scipy.ndimage import rotate
from scipy.misc import face
import PIL
from PIL import Image
from IPython.display import clear_output
import socket
import threading
import os, sys
import random

%matplotlib inline

start_position = numpy.array([
                    ['♜','♞','♝','♛','♚','♝','♞','♜'],
                    ['♟','♟','♟','♟','♟','♟','♟','♟'],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    [' ',' ',' ',' ',' ',' ',' ',' '],
                    ['♙','♙','♙','♙','♙','♙','♙','♙'],
                    ['♖','♘','♗','♕','♔','♗','♘','♖']])

reverse_start_position = numpy.rot90(start_position, 2)
path = "D:\Catalog\Проекты\Web\OpenServer\OSPanel\domains\Chess\icons/"

def resize():
    for item in os.listdir(path):
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((75,75), Image.ANTIALIAS)
            imResize.save(f + '.png', 'PNG', quality=100)
            

def toString(pos):
    s = ""
    for arr in pos:
        for sym in arr:
            s += sym
    return s
def toPos(s):
    pos = numpy.array([[' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' '],
          [' ',' ',' ',' ',' ',' ',' ',' ']])
    for i in range(8):
        for j in range(8):
            pos[i][j] = s[8*i+j]
    return pos
def isWhite(fig):
    if fig in "♙♖♘♗♕♔":
        return True
    return False

def isBlack(fig):
    if fig in "♟♜♞♝♛♚":
        return True
    return False

In [2]:
class chess:
    def __init__(self, vis = True, start_pos = start_position, prop = [False, False, False, False, False, False, [7, 4], [0, 4], True, [], False, 0, 0], virt = False):
        self.pos = start_pos.copy()
        self.fig_values = {}
        self.resources = []
        self.training_data = []
        self.ways = []
        self.isOppositeField = []
        self.isCheckField = []
        self.current_fields = []
        self.opposite_fields = []
        self.current_steps = []
        self.opposite_steps = []
        self.current_figs = []
        self.opposite_figs = []
        self.vis = vis
        self.white_king = prop[0]
        self.white_rook_right = prop[1]
        self.white_rook_left = prop[2]
        self.black_king = prop[3]
        self.black_rook_right = prop[4]
        self.black_rook_left = prop[5]
        self.white_king_coords = prop[6]
        self.black_king_coords = prop[7]
        self.white = prop[8]
        self.history = prop[9]
        self.isChecked = prop[10]
        self.count_check_figs = prop[11]
        self.repeat = prop[12]
        self.last = [[-1, -1], [-1, -1]]
        self.prop = prop
        self.MAX_STEPS = 1000
        self.isVirtual = virt
        if not virt:  
            self.updateFigs()
            self.updateSteps()
            self.initFigValues()
        if vis:
            print(self.pos)
    def initFigValues(self):
        self.fig_values['♙'] = 1.0
        self.fig_values['♟'] = 1.0
        self.fig_values['♘'] = 2.0
        self.fig_values['♞'] = 2.0
        self.fig_values['♗'] = 2.0
        self.fig_values['♝'] = 2.0
        self.fig_values['♖'] = 3.0
        self.fig_values['♜'] = 3.0
        self.fig_values['♕'] = 7.0
        self.fig_values['♛'] = 7.0
        self.fig_values['♔'] = 8.0
        self.fig_values['♚'] = 8.0
    def isWhite(self, v):
        if isWhite(self.pos[v[0]][v[1]]):
            return True
        return False
    def isBlack(self, v):
        if isBlack(self.pos[v[0]][v[1]]):
            return True
        return False
    def containsFig(self, reg, v):
        for c in reg:
            if self.pos[v[0]][v[1]] == c:
                return True
        return False
    def getFigs(self, reg):
        res = []
        for i in range(0, 8):
            for j in range(0, 8):
                if self.pos[i][j] != ' ' and self.pos[i][j] in reg:
                    res.append([i, j])
        return res
    def getCoefficient(self):
        c1 = 0.0
        c2 = 0.0
        for i in range(0, 8):
            for j in range(0, 8):
                if isWhite(self.board([i, j])):
                    c1 += self.fig_values[self.board([i, j])]
                elif isBlack((self.board([i, j]))):
                    c2 += self.fig_values[self.board([i, j])]
        return c1/(c1+c2)
    def setMaxSteps(self, steps):
        self.MAX_STEPS = steps
    def getPos(self):
        return self.pos
    def getReversePos(self):
        return numpy.rot90(self.pos, 2)
    def board(self, cur):
        return self.pos[cur[0]][cur[1]]
    def toArray(self):
        res = numpy.array([0.01]*82*7)
        i = 0
        figs = "♙♖♘♗♕♟♜♞♝♛"
        for k in range(0, len(figs)):
            i = 8*7*k
            c = self.getFigs(figs[k])
            for [x, y] in c:
                b = "{0:b}".format(8*x+y+1)
                for j in range(0, len(b)):
                    if b[j] == '1':
                        res[i] = 0.99
                    i+=1
                    j+=1
        i = 80*7
        for [x, y] in self.getFigs('♔'):
            b = "{0:b}".format(8*x+y+1)
            for j in range(0, len(b)):
                if b[j] == '1':
                    res[i] = 0.99
                i+=1
                j+=1
        i = 81*7
        for [x, y] in self.getFigs('♚'):
            b = "{0:b}".format(8*x+y+1)
            for j in range(0, len(b)):
                if b[j] == '1':
                    res[i] = 0.99
                i+=1
                j+=1
        return numpy.array(res)
    def isEqualColor(self, fr, to):
        fig_fr = self.board(fr)
        fig_to = self.board(to)
        return (isWhite(fig_fr) and isWhite(fig_to)) or (isBlack(fig_to) and isBlack(fig_fr))
    def isDiff(self, fr, to):
        fig_fr = self.board(fr)
        fig_to = self.board(to)
        return (isWhite(fig_fr) and isBlack(fig_to)) or (isWhite(fig_to) and isBlack(fig_fr))
    def updateProp(self):
        self.prop = [self.white_king, self.white_rook_right, self.white_rook_left, self.black_king, self.black_rook_right, self.black_rook_left, self.white_king_coords, self.black_king_coords, self.white, self.history, self.isChecked, self.count_check_figs, self.repeat]
    def updateFigs(self):
        self.current_figs.clear()
        self.opposite_figs.clear()
        if self.white:
            self.current_figs.append(self.white_king_coords)
        else:
            self.current_figs.append(self.black_king_coords)
        for i in range(8):
            for j in range(8):
                if self.current_figs[0] == [i, j]:
                    continue
                if (self.white and self.isWhite([i, j])) or (not self.white and self.isBlack([i, j])):
                    self.current_figs.append([i, j])
                elif (not self.white and self.isWhite([i, j])) or (self.white and self.isBlack([i, j])):
                    self.opposite_figs.append([i, j])
    def possibleStep(self, fr, to, onlyAttackFields = False):
        fr = [fr[0], fr[1]]
        to = [to[0], to[1]]
        st = [fr, to]
        if fr[0] < 0 or fr[0] > 7 or to[0] < 0 or to[0] > 7 or fr[1] < 0 or fr[1] > 7 or to[1] < 0 or to[1] > 7:
            return False
        if onlyAttackFields:
            return True
        fig_fr = self.board(fr)
        fig_to = self.board(to)
        if self.isChecked and fig_fr != '♔' and fig_fr != '♚':
            if not self.isCheckField[to[0]][to[1]] or self.count_check_figs > 1:
                return False
        vec = [0, 0]
        if self.ways[fr[0]][fr[1]] != vec:
            vec = self.ways[fr[0]][fr[1]]
            if vec[0] == 0:
                if fr[0] != to[0]:
                    return False
            elif vec[1] == 0:
                if fr[1] != to[1]:
                    return False
            else:
                i = (to[0]-fr[0])/vec[0]
                if to[1] != fr[1] + i*vec[1]:
                    return False
        last = [[-1, -1], [-1, -1]]
        if len(self.history) > 0:
            last = self.history[len(self.history)-1]
        if fig_fr == '♙':
            if fr[1] == to[1]:
                if to[0] == fr[0]-1 and fig_to == ' ':
                    return True
                if to[0] == fr[0]-2 and fig_to == ' ' and fr[0] == 6 and self.board((fr[0]-1, fr[1])) == ' ':
                    return True
            elif fig_to == ' ' and self.board((to[0]+1, to[1])) == '♟' and self.last[1] == [to[0]+1, to[1]] and self.last[0] == [to[0]-1, to[1]]:
                return True
            elif self.isDiff(fr, to):
                return True
            return False
            
        elif fig_fr == '♟':
            if fr[1] == to[1]:
                if to[0] == fr[0]+1 and self.board(to) == ' ':
                    return True
                if to[0] == fr[0]+2 and self.board(to) == ' ' and fr[0] == 1 and self.board((fr[0]+1, fr[1])) == ' ':
                    return True
            elif fig_to == ' ' and self.board((to[0]-1, to[1])) == '♙' and self.last[1] == [to[0]-1, to[1]] and self.last[0] == [to[0]+1, to[1]]:
                return True
            elif self.isDiff(fr, to):
                return True
            return False
            
        elif fig_fr == '♔' and abs(fr[1] - to[1]) >=2:
            if not self.white_rook_right and to[1]-fr[1] == 2 and to[1]-1 >= 0 and to[1]+1 < 8 and self.board((to[0], to[1]+1)) == '♖' and self.board((to[0], to[1]-1)) == ' ' and fig_to == ' ':
                if self.isOppositeField[to[0]][to[1]-1]:
                    return False
                if self.isOppositeField[to[0]][to[1]]:
                    return False
                return True
            elif not self.white_rook_left and fr[1]-to[1] == 2 and fr[1]-4 >= 0 and self.board((fr[0], fr[1]-4)) == '♖' and self.board((fr[0], fr[1]-3)) == ' ' and self.board((fr[0], fr[1]-2)) == ' ' and self.board((fr[0], fr[1]-1)) == ' ':
                if self.isOppositeField[fr[0]][fr[1]-2]:
                    return False
                if self.isOppositeField[fr[0]][fr[1]-1]:
                    return False
                return True
                
        elif fig_fr == '♚' and abs(fr[1] - to[1]) >=2:
            if not self.black_rook_right and to[1]-fr[1] == 2 and to[1]-1 >= 0 and to[1]+1 < 8 and self.board((to[0], to[1]+1)) == '♜' and self.board((to[0], to[1]-1)) == ' ' and fig_to == ' ':
                if self.isOppositeField[to[0]][to[1]-1]:
                    return False
                if self.isOppositeField[to[0]][to[1]]:
                    return False
                return True
            elif not self.black_rook_left and fr[1]-to[1] == 2 and fr[1]-4 >= 0 and self.board((fr[0], fr[1]-4)) == '♜' and self.board((fr[0], fr[1]-3)) == ' ' and self.board((fr[0], fr[1]-2)) == ' ' and self.board((fr[0], fr[1]-1)) == ' ':
                if self.isOppositeField[fr[0]][fr[1]-2]:
                    return False
                if self.isOppositeField[fr[0]][fr[1]-1]:
                    return False
                return True
                
        elif fig_to == ' ' or self.isDiff(fr, to):
            if fig_fr == '♔' or fig_fr == '♚':
                if self.isOppositeField[to[0]][to[1]]:
                    return False
            return True
        return False
    def getStepsByFig(self, cur, onlyAttackFields = False):
        fig = self.board(cur)
        steps = []
        if fig == '♙':
            if not onlyAttackFields:
                if self.possibleStep(cur, (cur[0]-1, cur[1])):
                    steps.append([cur, [cur[0]-1, cur[1]]])
                if self.possibleStep(cur, (cur[0]-2, cur[1])):
                    steps.append([cur, [cur[0]-2, cur[1]]])
            if self.possibleStep(cur, (cur[0]-1, cur[1]+1), onlyAttackFields):
                steps.append([cur, [cur[0]-1, cur[1]+1]])
                if onlyAttackFields and self.isDiff(cur, [cur[0]-1, cur[1]+1]):
                    if self.containsFig("♔♚", [cur[0]-1, cur[1]+1]):
                        self.isCheckField[cur[0]][cur[1]] = True
                        self.count_check_figs += 1
            if self.possibleStep(cur, (cur[0]-1, cur[1]-1), onlyAttackFields):
                steps.append([cur, [cur[0]-1, cur[1]-1]])
                if onlyAttackFields and self.isDiff(cur, [cur[0]-1, cur[1]-1]):
                    if self.containsFig("♔♚", [cur[0]-1, cur[1]-1]):
                        self.isCheckField[cur[0]][cur[1]] = True
                        self.count_check_figs += 1
        elif fig == '♟':
            if not onlyAttackFields:
                if self.possibleStep(cur, (cur[0]+1, cur[1])):
                    steps.append([cur, [cur[0]+1, cur[1]]])
                if self.possibleStep(cur, (cur[0]+2, cur[1])):
                    steps.append([cur, [cur[0]+2, cur[1]]])
            if self.possibleStep(cur, (cur[0]+1, cur[1]+1), onlyAttackFields):
                steps.append([cur, [cur[0]+1, cur[1]+1]])
                if onlyAttackFields and self.isDiff(cur, [cur[0]+1, cur[1]+1]):
                    if self.containsFig("♔♚", [cur[0]+1, cur[1]+1]):
                        self.isCheckField[cur[0]][cur[1]] = True
                        self.count_check_figs += 1
            if self.possibleStep(cur, (cur[0]+1, cur[1]-1), onlyAttackFields):
                steps.append([cur, [cur[0]+1, cur[1]-1]])
                if onlyAttackFields and self.isDiff(cur, [cur[0]+1, cur[1]-1]):
                    if self.containsFig("♔♚", [cur[0]+1, cur[1]-1]):
                        self.isCheckField[cur[0]][cur[1]] = True
                        self.count_check_figs += 1
        elif fig in '♖♜' or fig in '♗♝' or fig in '♕♛':
            inds = []
            if fig in '♖♜':
                inds = [[1, 0], [0, 1], [-1, 0], [0, -1]]
            elif fig in '♗♝':
                inds = [[1, 1], [-1, -1], [-1, 1], [1, -1]]
            else:
                inds = [[1, 0], [0, 1], [-1, 0], [0, -1], [1, 1], [-1, -1], [-1, 1], [1, -1]]
            for v in inds:
                a = v[0]
                b = v[1]
                while True:
                    if cur[0]+a < 0 or cur[0]+a > 7 or cur[1]+b < 0 or cur[1] + b > 7:
                        break
                    can = self.possibleStep(cur, [cur[0]+a, cur[1]+b], onlyAttackFields)
                    if self.isChecked:
                        if can:
                            steps.append([cur, [cur[0]+a, cur[1]+b]])
                        elif self.pos[cur[0]+a][cur[1]+b] != ' ':
                            break
                    elif not can:
                        break
                    else:
                        steps.append([cur, [cur[0]+a, cur[1]+b]])
                    if self.isDiff(cur, [cur[0]+a, cur[1]+b]):
                        king = False
                        if onlyAttackFields and self.containsFig('♔♚', [cur[0]+a, cur[1]+b]):
                            king = True
                            c = a
                            d = b
                            while True:
                                if c == v[0] and d == v[1]:
                                    break
                                c -= v[0]
                                d -= v[1]
                                self.isCheckField[cur[0]+c][cur[1]+d] = True
                            self.isCheckField[cur[0]][cur[1]] = True
                            self.count_check_figs += 1
                        if not king:
                            break
                    if onlyAttackFields and self.isEqualColor(cur, [cur[0]+a, cur[1]+b]):
                        break
                    a += v[0]
                    b += v[1]
                    
        elif fig in "♘♞":
            inds = [[-1, -2], [-1, 2], [1, -2], [1, 2], [-2, -1], [-2, 1], [2, -1], [2, 1]]
            for v in inds:
                if self.possibleStep(cur, [cur[0]+v[0], cur[1]+v[1]], onlyAttackFields):
                    steps.append([cur, [cur[0]+v[0], cur[1]+v[1]]])
                    if onlyAttackFields and self.isDiff(cur, [cur[0]+v[0], cur[1]+v[1]]):
                        if self.containsFig('♔♚', [cur[0]+v[0], cur[1]+v[1]]):
                            self.isCheckField[cur[0]][cur[1]] = True
                            self.count_check_figs += 1
        elif fig in "♔♚":
            inds = [[1, 0], [0, 1], [-1, 0], [0, -1], [1, 1], [-1, -1], [-1, 1], [1, -1]]
            for v in inds:
                if self.possibleStep(cur, [cur[0]+v[0], cur[1]+v[1]], onlyAttackFields):
                    steps.append([cur, [cur[0]+v[0], cur[1]+v[1]]])
            if not onlyAttackFields:
                if not self.isChecked:
                    if (fig == '♚' and not self.black_king) or (fig == '♔' and not self.white_king):
                        if self.possibleStep(cur, [cur[0], cur[1]+2]):
                            steps.append([cur, [cur[0], cur[1]+2]])
                        if self.possibleStep(cur, [cur[0], cur[1]-2]):
                            steps.append([cur, [cur[0], cur[1]-2]])
                
                for v in inds:
                    a = v[0]
                    b = v[1]
                    empty = [-1, -1]
                    found_fig = empty
                    ch = False
                    while True:
                        to = [cur[0]+a, cur[1]+b]
                        if to[0] < 0 or to[0] > 7 or to[1] < 0 or to[1] > 7:
                            break
                        if self.isEqualColor(cur, to):
                            if found_fig != empty:
                                break
                            found_fig = to
                        elif self.isDiff(cur, to):
                            if found_fig == empty:
                                break
                            if abs(v[0])+abs(v[1]) == 1:
                                if self.containsFig('♖♜♕♛', to):
                                    ch = True
                            else:
                                if self.containsFig('♗♝♕♛', to):
                                    ch = True
                            break
                        a += v[0]
                        b += v[1]
                    if ch and found_fig != empty:
                        self.ways[found_fig[0]][found_fig[1]] = v           
        return steps
    def updateSteps(self):
        self.current_steps.clear()
        self.opposite_steps.clear()
        self.current_fields.clear()
        self.opposite_fields.clear()
        self.isOppositeField = [[False for i in range(0, 8)] for j in range(0, 8)]
        self.isCheckField = [[False for i in range(0, 8)] for j in range(0, 8)]
        self.ways = [[[0, 0] for i in range(0, 8)] for j in range(0, 8)]
        self.isChecked = False
        self.count_check_figs = 0
        used = [[False for i in range(0, 8)] for j in range(0, 8)]
        
        for v in self.opposite_figs:
            got_steps = self.getStepsByFig(v, True)
            for st in got_steps:
                self.opposite_steps.append(st)
        self.isChecked = (self.count_check_figs > 0)
        for st in self.opposite_steps:
            if not used[st[1][0]][st[1][1]]:
                self.opposite_fields.append(st[1])
                self.isOppositeField[st[1][0]][st[1][1]] = True
                used[st[1][0]][st[1][1]] = True
        used = [[False for i in range(0, 8)] for j in range(0, 8)]
        
        for v in self.current_figs:
            got_steps = self.getStepsByFig(v, False)
            for st in got_steps:
                self.current_steps.append(st)
        for st in self.current_steps:
            if not used[st[1][0]][st[1][1]]:
                self.current_fields.append(st[1])
                used[st[1][0]][st[1][1]] = True
    def isCheck(self):
        if self.isChecked:
            return True
        return False
    def isMate(self):
        if self.isCheck():
            if len(self.current_steps) == 0:
                return True
        return False
    def isDraw(self):
        if not self.isCheck():
            if len(self.current_steps) == 0:
                return True
        return False
    def getStatus(self):
        if self.repeat >= 6 or len(self.resources) > self.MAX_STEPS or (len(self.current_figs) == 1 and len(self.opposite_figs) == 1):
            self.current_steps.clear()
            return "draw"
        if self.isDraw():
            return "draw"
        if self.isMate():
            if self.white:
                return "black"
            return "white"
        return "ingame"
    def isValidStep(self, fr, to):
        for cur in self.current_steps:
            if cur == [fr, to]:
                return True
        return False
    def doStep(self, fr, to):
        if self.isVirtual or self.isValidStep(fr, to):
            st = [fr, to]
            temp = self.board(fr)
            temp_to = self.board(to)
            self.pos[fr[0]][fr[1]] = ' '
            self.pos[to[0]][to[1]] = temp
            if temp in "♙♟":
                if to[0] == 0 or to[0] == 7:
                    if temp == '♙':
                        self.pos[to[0]][to[1]] = '♕'
                    else:
                        self.pos[to[0]][to[1]] = '♛'
                elif abs(to[0]-fr[0]) == 1 and abs(to[1]-fr[1]) == 1 and temp_to == ' ':
                    if temp == '♙':
                        self.pos[to[0]+1][to[1]] = ' '
                    else:
                        self.pos[to[0]-1][to[1]] = ' '
                        
            elif temp == '♔':
                if not self.white_king:
                    if to[1]-fr[1] == 2:
                        temp2 = self.board((to[0], to[1]+1))
                        self.pos[to[0]][to[1]+1] = ' '
                        self.pos[to[0]][to[1]-1] = temp2
                    elif fr[1]-to[1] == 2:
                        temp2 = self.board((fr[0], fr[1]-4))
                        self.pos[fr[0]][fr[1]-4] = ' '
                        self.pos[to[0]][to[1]+1] = temp2
                self.white_king = True
                self.white_king_coords = [to[0], to[1]]
            elif temp == '♚':
                if not self.black_king:
                    if to[1]-fr[1] == 2:
                        temp2 = self.board((to[0], to[1]+1))
                        self.pos[to[0]][to[1]+1] = ' '
                        self.pos[to[0]][to[1]-1] = temp2
                    elif fr[1]-to[1] == 2:
                        temp2 = self.board((fr[0], fr[1]-4))
                        self.pos[fr[0]][fr[1]-4] = ' '
                        self.pos[to[0]][to[1]+1] = temp2
                self.black_king = True
                self.black_king_coords = [to[0], to[1]]
            elif temp == '♖':
                if fr == [7, 0]:
                    self.white_rook_left = True
                elif fr == [7, 7]:
                    self.white_rook_right = True
            elif temp == '♜':
                if fr == [0, 0]:
                    self.black_rook_left = True
                elif fr == [0, 7]:
                    self.black_rook_right = True
            
            self.white = not self.white
            self.updateProp()
            self.updateFigs()
            if not self.isVirtual:
                if len(self.history) > 3:
                    if st == self.history[len(self.history)-4]:
                        self.repeat+=1
                    else:
                        self.repeat = 0
                self.history.append(st)
                self.last = st
                self.updateSteps()
                self.training_data.append(self.toArray())
                self.resources.append(self.getCoefficient())
                
            if self.vis:
                clear_output()
                print(self.pos)
                status = self.getStatus()
                if status != "ingame":
                    print(status)
            return True
        return False
    def doStepAI(self):
        eps = 0.15
        arr = []
        for st in self.current_steps:
            virt_game = chess(False, self.pos, self.prop, True)
            virt_game.doStep(st[0], st[1])
            inputs = virt_game.toArray()
            outputs = AI.query(inputs)
            Q = outputs[0][0]
            arr.append([[Q, st], inputs])
        if len(arr) > 0:
            best_steps = []
            arr.sort()
            if self.white:
                arr.reverse()
            for p in arr:
                if abs(p[0][0]-arr[0][0][0] > eps):
                    break
                best_steps.append(p)
            i = random.randint(0, len(best_steps)-1)
            self.doStep(best_steps[i][0][1][0], best_steps[i][0][1][1])
            return best_steps[i][0][1]
        return False
    def trainAI(self):
        status = self.getStatus()
        K = 0.5
        if status == "white":
            K = 0.99
        elif status == "black":
            K = 0.01
        for i in range(0, len(self.training_data)):
            AI.train(self.training_data[i], [(self.resources[i]+K)/2.0])

In [3]:
class neuralNetwork:
    def __init__(self, nodes, lr):
        self.nodes = nodes
        self.lr = lr
        self.layers = len(nodes)
        
        self.W = [numpy.random.normal(0.0, pow(self.nodes[i+1], -0.5), (self.nodes[i+1], self.nodes[i])) for i in range(0, self.layers-1)]
        
        self.activation_function = lambda x: scipy.special.expit(x)
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        outputs = inputs
        outputs_array = [outputs]
        
        for i in range(0, self.layers-1):
            outputs = self.activation_function(numpy.dot(self.W[i], outputs))
            outputs_array.append(outputs)
        
        errors = targets-outputs
        
        for i in range(self.layers-2, -1, -1):
            self.W[i] += self.lr * ((errors * outputs_array[i+1] * (1.0 - outputs_array[i+1]))) * (numpy.transpose(outputs_array[i]))
            errors = numpy.dot(self.W[i].T, errors)
        
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        outputs = inputs
        
        for i in range(0, self.layers-1):
            outputs = self.activation_function(numpy.dot(self.W[i], outputs))
        
        return outputs
    def saveW(self):
        for i in range(self.layers-1):
            numpy.savetxt("weights/W"+str(i+1), self.W[i])
    def loadW(self):
        for i in range(self.layers-1):
            self.W[i] = numpy.loadtxt("weights/W"+str(i+1))
            if len(self.W[i].shape) == 1:
                self.W[i] = numpy.array([self.W[i]])
    def clearW(self):
        self.W = [numpy.random.normal(0.0, pow(self.nodes[i+1], -0.5), (self.nodes[i+1], self.nodes[i])) for i in range(0, self.layers-1)]

In [4]:
#Данные нейросети
nodes = [82*7, 300, 1]
learning_rate = 0.1

AI = neuralNetwork(nodes, learning_rate)
AI.loadW()

In [ ]:
#Реализация сервера (variant 1 - for Web)
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "192.168.0.103"
port = 1234
soc.bind((host, port))
soc.listen(10)

USERS_me = dict()
USERS_player = dict()
USERS_ai = dict()

class User(threading.Thread):
    def __init__(self, num, con):
        super().__init__(target=None, name=("TH"+num), args=(con,))
        self.num = num
        self.con = con
    def run(self):
        while(True):
            try:
                data = self.con.recv(1024).decode()
                print(data)
                arr = data.split("|")
                if len(arr) == 4 and arr[3] == "remove":
                    #print(data)
                    username = arr[0]
                    who = arr[1]
                    typ = arr[2]
                    if who == "me":
                        USERS_me[username] = [chess(False), typ, "ingame"]
                        if USERS_me[username][1] == "white":
                            self.con.send(toString(USERS_me[username][0].getPos()).encode('utf-8'))
                        else:
                            self.con.send(toString(USERS_me[username][0].getReversePos()).encode('utf-8'))
                    elif who == "ai":
                        USERS_ai[username] = [chess(False), typ, "ingame", [(-1, -1), (-1, -1)]]
                        if USERS_ai[username][1] == "white":
                            self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                        else:
                            self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                if ',' not in data:
                    username = arr[0]
                    who = arr[1]
                    if who == "me":
                        typ = arr[2]
                        if username not in USERS_me:
                            USERS_me[username] = [chess(False), typ, "ingame"]
                        USERS_me[username][1] = typ
                        status = USERS_me[username][2]
                        if USERS_me[username][1] == "white":
                            self.con.send((toString(USERS_me[username][0].getPos())+"|"+status).encode('utf-8'))
                        else:
                            self.con.send((toString(USERS_me[username][0].getReversePos())+"|"+status).encode('utf-8'))
                            
                        if USERS_me[username][2] != "ingame":
                            USERS_me[username] = [chess(False), typ, "ingame"]
                    elif who == "ai":
                        ul = arr[2]
                        if username not in USERS_ai:
                            rand = random.randrange(0, 2)
                            typ = "white"
                            if rand == 1:
                                typ = "black"
                            USERS_ai[username] = [chess(False), typ, "ingame", [[-1, -1], [-1, -1]]]
                        
                        if USERS_ai[username][3] != USERS_ai[username][0].last or ul == "load":
                            print(data)
                            USERS_ai[username][3] = USERS_ai[username][0].last
                            status = USERS_ai[username][2]
                            if USERS_ai[username][1] == "white":
                                self.con.send((toString(USERS_ai[username][0].getPos())+"|"+status+"|"+USERS_ai[username][1]).encode('utf-8'))
                            else:
                                if USERS_ai[username][3] == [[-1, -1], [-1, -1]]:
                                    USERS_ai[username][0].doStepAI()
                                self.con.send((toString(USERS_ai[username][0].getReversePos())+"|"+status+"|"+USERS_ai[username][1]).encode('utf-8'))
                        else:
                            self.con.send("No change".encode('utf-8'))
                        if USERS_ai[username][2] != "ingame":
                            USERS_ai[username][0].trainAI()
                            AI.saveW()
                            rand = random.randrange(0, 2)
                            typ = "white"
                            if rand == 1:
                                typ = "black"
                            USERS_ai[username] = [chess(False), typ, "ingame", [(-1, -1), (-1, -1)]]
                    elif who == "player":
                        what = arr[2]
                else:
                    #print(data)
                    username = arr[0]
                    who = arr[1]
                    arr_step = arr[2].split(",")
                    
                    a = int(arr_step[0])
                    b = int(arr_step[1])
                    c = int(arr_step[2])
                    d = int(arr_step[3])
                    
                    if who == "me":
                        if USERS_me[username][0].doStep([a,b], [c,d]):
                            status = USERS_me[username][0].getStatus()
                            if status == "ingame":
                                if USERS_me[username][1] == "white":
                                    self.con.send(toString(USERS_me[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_me[username][0].getReversePos()).encode('utf-8'))
                            else:
                                print(status)
                                USERS_me[username][2] = status
                                self.con.send(status.encode('utf-8'))
                        else:
                            print("impossible")
                            self.con.send("impossible".encode('utf-8'))
                    elif who == "ai":
                        if USERS_ai[username][0].doStep([a,b], [c,d]):
                            status = USERS_ai[username][0].getStatus()
                            if status == "ingame":
                                if USERS_ai[username][1] == "white":
                                    self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                                
                                USERS_ai[username][0].doStepAI()
                                if USERS_ai[username][1] == "white":
                                    self.con.send(toString(USERS_ai[username][0].getPos()).encode('utf-8'))
                                else:
                                    self.con.send(toString(USERS_ai[username][0].getReversePos()).encode('utf-8'))
                            else:
                                print(status)
                                USERS_ai[username][2] = status
                                self.con.send(status.encode('utf-8'))
                        else:
                            print("impossible")
                            self.con.send("impossible".encode('utf-8'))
                        
            except:
                print("Пользователь отключился")
                return

while(True):
    conn, addr = soc.accept()
    print("Пользователь подключился: ",addr)
    TH = User(str(addr), conn)
    TH.start()

Пользователь подключился:  ('192.168.43.1', 58362)
Алексей Ахтямов|me|white


In [ ]:
#Реализация сервера (variant 2 - for Android)
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "192.168.43.39"
port = 1234
soc.bind((host, port))
soc.listen(10)

USERS = dict()
USERS['me'] = dict()
USERS['players'] = dict()
USERS['ai'] = dict()

class User(threading.Thread):
    def __init__(self, num, con):
        super().__init__(target=None, name=("TH"+num), args=(con,))
        self.num = num
        self.con = con
    def run(self):
        data = self.con.recv(1024).decode()
        print(data)
        inf = data.split("|")
        
        action = inf[0]
        username = inf[1]
        type_game = inf[2]
        
        if action == "user_add":
            who = inf[3]
            if username not in USERS[type_game]:
                USERS[type_game][username] = chess(False)
                if who == "black" and type_game == "ai":
                    USERS[type_game][username].doStepAI()
        elif action == "user_remove":
            if username in USERS[type_game]:
                del USERS[type_game][username]
        elif action == "do_step":
            step = inf[3].split(",")
            fr = [int(step[0]), int(step[1])]
            to = [int(step[2]), int(step[3])]
            if username in USERS[type_game]:
                if USERS[type_game][username].doStep(fr, to):
                    status = USERS[type_game][username].getStatus()
                    self.con.send(status.encode('utf-8'))
                    if type_game == "ai":
                        USERS[type_game][username].doStepAI()
                else:
                    self.con.send("incorrect".encode('utf-8'))
                    print("incorrect")
            else:
                self.con.send("incorrect".encode('utf-8'))
        elif action == "get_pos":
            if username in USERS[type_game]:
                self.con.send(toString(USERS[type_game][username].getPos()).encode('utf-8'))

while(True):
    conn, addr = soc.accept()
    print("Пришёл пакет от: ",addr)
    TH = User(str(addr), conn)
    TH.start()

Пришёл пакет от:  ('192.168.43.1', 46908)
user_add|player|ai|white
Пришёл пакет от:  ('192.168.43.1', 46910)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46912)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46914)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46916)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46918)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46920)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46922)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46924)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46926)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46928)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46930)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46932)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46934)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46936)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1', 46938)
get_pos|player|ai
Пришёл пакет от:  ('192.168.43.1'

Пришёл пакет от:  ('192.168.43.1', 47174)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47176)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47178)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47180)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47182)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47184)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47186)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47188)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47190)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47192)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47194)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47196)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47198)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47200)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47202)
get_pos|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47204)
user_remove|WWW|ai
Пришёл пакет от:  ('192.168.43.1', 47206)
user_remove|WWW|ai
Пришёл пакет от:  ('192

Пришёл пакет от:  ('192.168.43.1', 47444)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47446)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47448)
do_step|liza lox|me|7,5,6,4
Пришёл пакет от:  ('192.168.43.1', 47450)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47452)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47454)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47456)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47458)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47460)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47462)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47464)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47466)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47468)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47470)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47472)
get_pos|liza lox|me
Пришёл пакет от:  ('192.168.43.1', 47474)
do_step|liza lox|me|

Пришёл пакет от:  ('192.168.43.1', 47710)
do_step|pipy|me|5,4,1,5
incorrect
Пришёл пакет от:  ('192.168.43.1', 47712)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47714)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47716)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47718)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47720)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47722)
do_step|pipy|me|1,5,2,5
Пришёл пакет от:  ('192.168.43.1', 47724)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47726)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47728)
do_step|pipy|me|2,5,7,6
incorrect
Пришёл пакет от:  ('192.168.43.1', 47730)
do_step|pipy|me|7,6,5,5
Пришёл пакет от:  ('192.168.43.1', 47732)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47734)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47736)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47738)
get_pos|pipy|me
Пришёл пакет от:  ('192.168.43.1', 47740)
get_pos|pipy|me
Пришёл пакет от:  ('

Пришёл пакет от:  ('192.168.43.1', 47964)
get_pos|Tu it to|ai
